In [1]:
import json

In [2]:
with open("medical_zh/train_zh_0.jsonl", "r") as f:
    lst = [json.loads(item) for item in f.readlines()[:1000]]

In [3]:
lst[0]

{'instruction': '血热的临床表现是什么?',
 'input': '',
 'output': '初发或复发病不久。皮疹发展迅速，呈点滴状、钱币状或混合状。常见丘疹、斑丘疹、大小不等的斑片，潮红、鲜红或深红色。散布于体表各处或几处，以躯干、四肢多见，亦可先从头面开始，逐渐发展至全身。新皮疹不断出现，表面覆有银白色鳞屑，干燥易脱落，剥刮后有点状出血。可有同形反应;伴瘙痒、心烦口渴。大便秘结、小便短黄，舌质红赤，苔薄黄或根部黄厚，脉弦滑或滑数。血热炽盛病机，主要表现在如下四个面：一、热象：血热多属阳盛则热之实性、热性病机和病证、并表现出热象。二、血行加速：血得热则行，可使血流加速，且使脉道扩张，络脉充血，故可见面红目赤，舌色深红（即舌绛）等症。三、动血：在血行加速与脉道扩张的基础上，血分有热，可灼伤脉络，引起出血，称为“热迫血妄行”，或称动血。四、扰乱心神：血热炽盛则扰动心神，心主血脉而藏神，血脉与心相通，故血热则使心神不安，而见心烦，或躁扰发狂等症。'}

In [4]:
with open("train_lora.json", "w") as f:
    json.dump(lst, f, ensure_ascii=False)

In [5]:
import pandas as pd
import datasets

df = pd.read_json('train_lora.json')
ds = datasets.Dataset.from_pandas(df)

/home/pai/envs/qwen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
ds[0]

{'instruction': '血热的临床表现是什么?',
 'input': '',
 'output': '初发或复发病不久。皮疹发展迅速，呈点滴状、钱币状或混合状。常见丘疹、斑丘疹、大小不等的斑片，潮红、鲜红或深红色。散布于体表各处或几处，以躯干、四肢多见，亦可先从头面开始，逐渐发展至全身。新皮疹不断出现，表面覆有银白色鳞屑，干燥易脱落，剥刮后有点状出血。可有同形反应;伴瘙痒、心烦口渴。大便秘结、小便短黄，舌质红赤，苔薄黄或根部黄厚，脉弦滑或滑数。血热炽盛病机，主要表现在如下四个面：一、热象：血热多属阳盛则热之实性、热性病机和病证、并表现出热象。二、血行加速：血得热则行，可使血流加速，且使脉道扩张，络脉充血，故可见面红目赤，舌色深红（即舌绛）等症。三、动血：在血行加速与脉道扩张的基础上，血分有热，可灼伤脉络，引起出血，称为“热迫血妄行”，或称动血。四、扰乱心神：血热炽盛则扰动心神，心主血脉而藏神，血脉与心相通，故血热则使心神不安，而见心烦，或躁扰发狂等症。'}

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

ckpt = 'Qwen-1_8B-Chat'
tokenizer = AutoTokenizer.from_pretrained(ckpt, use_fast=False, trust_remote_code=True)
tokenizer.pad_token_id = tokenizer.eod_id
tokenizer

QWenTokenizer(name_or_path='Qwen-1_8B-Chat', vocab_size=151851, model_max_length=8192, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	
}

In [8]:
def process_func(example):
    MAX_LENGTH = 384    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["<|im_start|>system", "你是一个医学助手，需要回答用户关于医学的问题：<|im_end|>" + "\n<|im_start|>user\n" + example["instruction"] + example["input"] + "<|im_end|>\n"]).strip(), add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer("<|im_start|>assistant\n" + example["output"] + "<|im_end|>\n", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  # Qwen的特殊构造就是这样的
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [9]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map: 100%|██████████| 1000/1000 [00:00<00:00, 2442.86 examples/s]


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})

In [10]:
tokenizer.decode(tokenized_id[1]['input_ids'])

'<|im_start|>system\n你是一个医学助手，需要回答用户关于医学的问题：<|im_end|>\n<|im_start|>user\n帕金森叠加综合征的辅助治疗有些什么？<|im_end|><|im_start|>assistant\n综合治疗；康复训练；生活护理指导；低频重复经颅磁刺激治疗<|im_end|>\n<|endoftext|>'

In [11]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"])))

'<|im_start|>assistant\n综合治疗；康复训练；生活护理指导；低频重复经颅磁刺激治疗<|im_end|>\n<|endoftext|>'

In [12]:
import torch

model = AutoModelForCausalLM.from_pretrained(ckpt, trust_remote_code=True, torch_dtype=torch.half, load_in_8bit=True, device_map="auto")
model

The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.54it/s]


QWenLMHeadModel(
  (transformer): QWenModel(
    (wte): Embedding(151936, 2048)
    (drop): Dropout(p=0.0, inplace=False)
    (rotary_emb): RotaryEmbedding()
    (h): ModuleList(
      (0-23): 24 x QWenBlock(
        (ln_1): RMSNorm()
        (attn): QWenAttention(
          (c_attn): Linear8bitLt(in_features=2048, out_features=6144, bias=True)
          (c_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
          (attn_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): RMSNorm()
        (mlp): QWenMLP(
          (w1): Linear8bitLt(in_features=2048, out_features=5504, bias=False)
          (w2): Linear8bitLt(in_features=2048, out_features=5504, bias=False)
          (c_proj): Linear8bitLt(in_features=5504, out_features=2048, bias=False)
        )
      )
    )
    (ln_f): RMSNorm()
  )
  (lm_head): Linear(in_features=2048, out_features=151936, bias=False)
)

In [13]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [14]:
model.dtype

torch.float16

In [15]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["c_attn", "c_proj", "w1", "w2"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'w1', 'c_proj', 'w2', 'c_attn'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [16]:
model = get_peft_model(model, config)
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): QWenLMHeadModel(
      (transformer): QWenModel(
        (wte): Embedding(151936, 2048)
        (drop): Dropout(p=0.0, inplace=False)
        (rotary_emb): RotaryEmbedding()
        (h): ModuleList(
          (0-23): 24 x QWenBlock(
            (ln_1): RMSNorm()
            (attn): QWenAttention(
              (c_attn): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=2048, out_features=6144, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=6144, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
     

In [17]:
model.print_trainable_parameters()

trainable params: 6,709,248 || all params: 1,843,537,920 || trainable%: 0.3639


In [18]:
args = TrainingArguments(
    output_dir=f"./output/{ckpt}_old",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=12,
    gradient_checkpointing=True,
    save_steps=186,
    learning_rate=1e-4,
)

In [19]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

/home/pai/envs/qwen/lib/python3.10/site-packages/accelerate/accelerator.py:444: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
Detected kernel version 4.19.24, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [20]:
trainer.train()

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/pai/envs/qwen/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.584600
20,2.115600
30,2.163300
40,2.012300
50,2.048600
60,2.035200
70,1.911900
80,1.925900
90,1.899100
100,1.920000


/home/pai/envs/qwen/lib/python3.10/site-packages/peft/utils/other.py:611: UserWarning: Unable to fetch remote file due to the following error (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /Qwen-1_8B-Chat/resolve/main/config.json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f4d9af09c30>: Failed to establish a new connection: [Errno 101] Network is unreachable'))"), '(Request ID: d91a59da-6f18-4cb7-a80d-6d88f4b2456b)') - silently ignoring the lookup for the file config.json in Qwen-1_8B-Chat.
  warnings.warn(
/home/pai/envs/qwen/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in Qwen-1_8B-Chat - will assume that the vocabulary was not modified.
  warnings.warn(
/home/pai/envs/qwen/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2

TrainOutput(global_step=744, training_loss=1.3652763059062343, metrics={'train_runtime': 2001.8862, 'train_samples_per_second': 5.994, 'train_steps_per_second': 0.372, 'total_flos': 3.072383330992128e+16, 'train_loss': 1.3652763059062343, 'epoch': 11.9})

In [21]:
response, history = model.chat(tokenizer, "帕金森叠加综合征的辅助治疗有些什么?", history=[], system="你是一个医学助手，需要回答用户关于医学的问题：")
response, history

/home/pai/envs/qwen/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/pai/envs/qwen/lib/python3.10/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[A1] query='帕金森叠加综合征的辅助治疗有些什么?' history=[] system='你是一个医学助手，需要回答用户关于医学的问题：' max_window_size=6144
[A2] raw_text='<|im_start|>system\n你是一个医学助手，需要回答用户关于医学的问题：<|im_end|>\n<|im_start|>user\n帕金森叠加综合征的辅助治疗有些什么?<|im_end|>\n<|im_start|>assistant\n' context_tokens=[151644, 8948, 198, 56568, 101909, 104316, 110498, 3837, 85106, 102104, 20002, 101888, 104316, 103936, 5122, 151645, 198, 151644, 872, 198, 105091, 34230, 100739, 110081, 118458, 9370, 104650, 101899, 101895, 99245, 30, 151645, 198, 151644, 77091, 198]

[B1] input_ids=tensor([[151644,   8948,    198,  56568, 101909, 104316, 110498,   3837,  85106,
         102104,  20002, 101888, 104316, 103936,   5122, 151645,    198, 151644,
            872,    198, 105091,  34230, 100739, 110081, 118458,   9370, 104650,
         101899, 101895,  99245,     30, 151645,    198, 151644,  77091,    198]],
       device='cuda:0') stop_words_ids=[[151645], [151644]] generation_config=GenerationConfig {
  "chat_format": "chatml",
  "do_sample": true,
  "eo

('低频重复经颅磁刺激治疗；配合综合康复治疗；低引产重复经颅磁刺激治疗(早期);配合综合康复治疗',
 [('帕金森叠加综合征的辅助治疗有些什么?', '低频重复经颅磁刺激治疗；配合综合康复治疗；低引产重复经颅磁刺激治疗(早期);配合综合康复治疗')])